<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Charlie", "transactions": [{"transaction-id": 489, "amount": 27}, {"transaction-id": 555, "amount": 29}, {"transaction-id": 617, "amount": 28}, {"transaction-id": 953, "amount": 35}, {"transaction-id": 1934, "amount": 30}, {"transaction-id": 2064, "amount": 33}, {"transaction-id": 2188, "amount": 29}, {"transaction-id": 2274, "amount": 28}, {"transaction-id": 2279, "amount": 29}, {"transaction-id": 2403, "amount": 27}, {"transaction-id": 2405, "amount": 32}, {"transaction-id": 2414, "amount": 34}, {"transaction-id": 2464, "amount": 31}, {"transaction-id": 2466, "amount": 29}, {"transaction-id": 2572, "amount": 31}, {"transaction-id": 2701, "amount": 30}, {"transaction-id": 2898, "amount": 33}, {"transaction-id": 3010, "amount": 31}, {"transaction-id": 3199, "amount": 29}, {"transaction-id": 3290, "amount": 32}, {"transaction-id": 3398, "amount": 30}, {"transaction-id": 3590, "amount": 29}, {"transaction-id": 3603, "amount": 35}, {"transaction-id": 3687, "amount": 2

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Charlie", "transactions": [{"transaction-id": 489, "amount": 27}, {"transaction-id": 555, "amount": 29}, {"transaction-id": 617, "amount": 28}, {"transaction-id": 953, "amount": 35}, {"transaction-id": 1934, "amount": 30}, {"transaction-id": 2064, "amount": 33}, {"transaction-id": 2188, "amount": 29}, {"transaction-id": 2274, "amount": 28}, {"transaction-id": 2279, "amount": 29}, {"transaction-id": 2403, "amount": 27}, {"transaction-id": 2405, "amount": 32}, {"transaction-id": 2414, "amount": 34}, {"transaction-id": 2464, "amount": 31}, {"transaction-id": 2466, "amount": 29}, {"transaction-id": 2572, "amount": 31}, {"transaction-id": 2701, "amount": 30}, {"transaction-id": 2898, "amount": 33}, {"transaction-id": 3010, "amount": 31}, {"transaction-id": 3199, "amount": 29}, {"transaction-id": 3290, "amount": 32}, {"transaction-id": 3398, "amount": 30}, {"transaction-id": 3590, "amount": 29}, {"transaction-id": 3603, "amount": 35}, {"transaction-id": 3687, "amount": 2

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Charlie',
  'transactions': [{'transaction-id': 489, 'amount': 27},
   {'transaction-id': 555, 'amount': 29},
   {'transaction-id': 617, 'amount': 28},
   {'transaction-id': 953, 'amount': 35},
   {'transaction-id': 1934, 'amount': 30},
   {'transaction-id': 2064, 'amount': 33},
   {'transaction-id': 2188, 'amount': 29},
   {'transaction-id': 2274, 'amount': 28},
   {'transaction-id': 2279, 'amount': 29},
   {'transaction-id': 2403, 'amount': 27},
   {'transaction-id': 2405, 'amount': 32},
   {'transaction-id': 2414, 'amount': 34},
   {'transaction-id': 2464, 'amount': 31},
   {'transaction-id': 2466, 'amount': 29},
   {'transaction-id': 2572, 'amount': 31},
   {'transaction-id': 2701, 'amount': 30},
   {'transaction-id': 2898, 'amount': 33},
   {'transaction-id': 3010, 'amount': 31},
   {'transaction-id': 3199, 'amount': 29},
   {'transaction-id': 3290, 'amount': 32},
   {'transaction-id': 3398, 'amount': 30},
   {'transaction-id': 3590, 'amount': 29},
   {'trans

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 113,
  'name': 'Alice',
  'transactions': [{'transaction-id': 197, 'amount': -265},
   {'transaction-id': 524, 'amount': -289},
   {'transaction-id': 610, 'amount': -277},
   {'transaction-id': 615, 'amount': -283},
   {'transaction-id': 820, 'amount': -271},
   {'transaction-id': 823, 'amount': -249},
   {'transaction-id': 1613, 'amount': -214},
   {'transaction-id': 1660, 'amount': -230},
   {'transaction-id': 2149, 'amount': -277},
   {'transaction-id': 2436, 'amount': -284},
   {'transaction-id': 2809, 'amount': -292},
   {'transaction-id': 2931, 'amount': -239},
   {'transaction-id': 3031, 'amount': -211},
   {'transaction-id': 3214, 'amount': -255},
   {'transaction-id': 3418, 'amount': -255},
   {'transaction-id': 3714, 'amount': -307},
   {'transaction-id': 4298, 'amount': -262},
   {'transaction-id': 4801, 'amount': -301},
   {'transaction-id': 5138, 'amount': -301},
   {'transaction-id': 5166, 'amount': -348},
   {'transaction-id': 5326, 'amount': -275},
   {'transact

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 49},
 {'name': 'Alice', 'count': 69},
 {'name': 'Alice', 'count': 80},
 {'name': 'Alice', 'count': 68})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(49, 69, 80, 68)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

67.4

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 197, 'amount': -265},
  {'transaction-id': 524, 'amount': -289},
  {'transaction-id': 610, 'amount': -277},
  {'transaction-id': 615, 'amount': -283},
  {'transaction-id': 820, 'amount': -271},
  {'transaction-id': 823, 'amount': -249},
  {'transaction-id': 1613, 'amount': -214},
  {'transaction-id': 1660, 'amount': -230},
  {'transaction-id': 2149, 'amount': -277},
  {'transaction-id': 2436, 'amount': -284},
  {'transaction-id': 2809, 'amount': -292},
  {'transaction-id': 2931, 'amount': -239},
  {'transaction-id': 3031, 'amount': -211},
  {'transaction-id': 3214, 'amount': -255},
  {'transaction-id': 3418, 'amount': -255},
  {'transaction-id': 3714, 'amount': -307},
  {'transaction-id': 4298, 'amount': -262},
  {'transaction-id': 4801, 'amount': -301},
  {'transaction-id': 5138, 'amount': -301},
  {'transaction-id': 5166, 'amount': -348},
  {'transaction-id': 5326, 'amount': -275},
  {'transaction-id': 5495, 'amount': -260},
  {'transaction-id': 5618, 'amount': -

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 197, 'amount': -265},
 {'transaction-id': 524, 'amount': -289},
 {'transaction-id': 610, 'amount': -277})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(-265, -289, -277)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

3204.248664688427

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 48), ('Alice', 48), ('Alice', 48), ('Alice', 56), ('Bob', 120), ('Bob', 120), ('Bob', 120), ('Bob', 140), ('Charlie', 156), ('Charlie', 156), ('Charlie', 156), ('Charlie', 182), ('Dan', 119), ('Dan', 120), ('Dan', 120), ('Dan', 139), ('Edith', 84), ('Edith', 84), ('Edith', 84), ('Edith', 98), ('Frank', 132), ('Frank', 132), ('Frank', 132), ('Frank', 154), ('George', 144), ('George', 144), ('George', 311), ('Hannah', 143), ('Hannah', 143), ('Hannah', 144), ('Hannah', 168), ('Ingrid', 72), ('Ingrid', 72), ('Ingrid', 72), ('Ingrid', 83), ('Jerry', 81), ('Jerry', 96), ('Jerry', 159), ('Kevin', 138), ('Kevin', 139), ('Kevin', 140), ('Kevin', 162), ('Laura', 96), ('Laura', 96), ('Laura', 96), ('Laura', 112), ('Michael', 124), ('Michael', 125), ('Michael', 129), ('Michael', 148), ('Norbert', 120), ('Norbert', 120), ('Norbert', 120), ('Norbert', 140), ('Oliver', 96), ('Oliver', 96), ('Oliver', 96), ('Oliver', 112), ('Patricia', 125), ('Patricia', 128), ('Patricia', 128), ('Patricia'

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 200), ('Bob', 500), ('Charlie', 650), ('Dan', 498), ('Edith', 350), ('Frank', 550), ('George', 599), ('Hannah', 598), ('Ingrid', 299), ('Jerry', 336), ('Kevin', 579), ('Laura', 400), ('Michael', 526), ('Norbert', 500), ('Oliver', 400), ('Patricia', 533), ('Quinn', 200), ('Ray', 591), ('Sarah', 567), ('Tim', 349), ('Ursula', 749), ('Victor', 550), ('Wendy', 350), ('Xavier', 250), ('Yvonne', 500), ('Zelda', 699)]
CPU times: user 124 ms, sys: 5.43 ms, total: 130 ms
Wall time: 433 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Charlie,"[{'transaction-id': 489, 'amount': 27}, {'tran..."
1,1,Ray,"[{'transaction-id': 37, 'amount': 1458}, {'tra..."
2,2,Patricia,"[{'transaction-id': 5, 'amount': 49}, {'transa..."
3,3,Yvonne,"[{'transaction-id': 131, 'amount': 232}, {'tra..."
4,4,Ursula,"[{'transaction-id': 190, 'amount': 989}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 236 ms, sys: 4.96 ms, total: 241 ms
Wall time: 1.32 s


name
Alice      200
Bob        500
Charlie    650
Dan        498
Edith      350
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Charlie', 'amount': 27, 'transaction-id': 489},
 {'id': 0, 'name': 'Charlie', 'amount': 29, 'transaction-id': 555},
 {'id': 0, 'name': 'Charlie', 'amount': 28, 'transaction-id': 617})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Charlie,27,489
1,0,Charlie,29,555
2,0,Charlie,28,617
3,0,Charlie,35,953
4,0,Charlie,30,1934


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 182 ms, sys: 4.65 ms, total: 187 ms
Wall time: 988 ms


name
Alice       13480
Bob         29303
Charlie     25743
Dan         15319
Edith       16001
Frank       23928
George      22833
Hannah      13884
Ingrid       9536
Jerry       19348
Kevin       28553
Laura       19271
Michael      8039
Norbert     19210
Oliver      22281
Patricia    20501
Quinn        9923
Ray         22279
Sarah       28463
Tim         17870
Ursula      33428
Victor      16719
Wendy       10309
Xavier      10507
Yvonne      19227
Zelda       24045
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/bag/core.py:2443: UserWarning: Insufficient elements for `take`. 5 elements requested, only 4 elements available. Try passing larger `npartitions` to `take`.
  warnings.warn(
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/bag/core.py:2443: UserWarning: Insufficient elements for `take`. 5 elements requested, only 4 elements available. Try passing larger `npartitions` to `take`.
  warnings.warn(
